### All needed imports

In [1]:
import os
import cv2
import numpy as np
import matplotlib as pl
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


### Limit GPU Usage

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-01 13:46:52.556560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 13:46:52.572367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 13:46:52.572510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 13:46:52.573171: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Preprocessing Data for Model 1

#### Video Paths

In [3]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [4]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
            
            
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [66]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [67]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
# checking data of still_diff
# for i, x in enumerate(still_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)

# checking data of still_diff
# for i, x in enumerate(falling_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]


# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

(1080, 224, 224, 3)


#### Create the model

In [68]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Convolutional layers for image processing
    # Convolutional layers are useful for learning spatial hierarchies and detecting features 
    x = Conv2D(32, (3, 3), activation='relu')(combined_input) #  Convolutional layers extract spatial features from the image
    x = MaxPooling2D((2, 2))(x) #  Max-pooling layers reduce the size of the feature maps.
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers: These layers learn to make decisions based on the features extracted by the convolutional layers
    x = Flatten()(x) # This line flattens the 2D feature maps into a 1D vector
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model

In [69]:
model = create_model()
model.summary()
model.fit(dataset, validation_data=val_dataset, epochs=5)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 128)       7385

Epoch 1/5
68/68 [==============================] - 3s 39ms/step - loss: 29.2882 - accuracy: 0.8519 - val_loss: 0.0197 - val_accuracy: 1.0000
Epoch 2/5
68/68 [==============================] - 2s 35ms/step - loss: 0.0461 - accuracy: 0.9889 - val_loss: 0.0078 - val_accuracy: 1.0000
Epoch 3/5
68/68 [==============================] - 3s 38ms/step - loss: 0.0557 - accuracy: 0.9824 - val_loss: 0.0136 - val_accuracy: 0.9954
Epoch 4/5
68/68 [==============================] - 3s 39ms/step - loss: 0.0750 - accuracy: 0.9815 - val_loss: 0.0148 - val_accuracy: 0.9907
Epoch 5/5
68/68 [==============================] - 3s 37ms/step - loss: 0.0332 - accuracy: 0.9889 - val_loss: 0.0044 - val_accuracy: 1.0000


In [17]:
# Testing something not part of the main code
arr1 = [[1,2,3], [1,2,3], [1,2,3]]
arr1 = np.array(arr1)
print(arr1.shape)
print(arr1)
print()
# exapand dims through axis 0
arr1_expanded = np.expand_dims(arr1, axis = 0)
print(arr1_expanded.shape)
print(arr1_expanded)
print()
# exapand dims through axis 1
arr2 = [[1,2,3], [1,2,3], [1,2,3]]
arr2 = np.array(arr2)
arr2_expanded = np.expand_dims(arr2, axis = 1)
print(arr2_expanded.shape)
print(arr2_expanded)
print()
arr3 = [[1,2,3], [1,2,3], [1,2,3]]
arr3 = np.array(arr3)
arr3_expanded = np.expand_dims(arr3, axis = -1)
print(arr3_expanded.shape)
print(arr3_expanded)

(3, 3)
[[1 2 3]
 [1 2 3]
 [1 2 3]]

(1, 3, 3)
[[[1 2 3]
  [1 2 3]
  [1 2 3]]]

(3, 1, 3)
[[[1 2 3]]

 [[1 2 3]]

 [[1 2 3]]]

(3, 3, 1)
[[[1]
  [2]
  [3]]

 [[1]
  [2]
  [3]]

 [[1]
  [2]
  [3]]]
